In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format',str)

In [ ]:
os.chdir("C:/Users/DELL/Pandas")

In [ ]:
import zipfile
import os

# Define the path to the zip file
zip_file_path = 'C:/Users/DELL/Pandas/travel.sqlite.zip'

# Create a directory to extract the database file
extracted_dir = 'extracted_database'
os.makedirs(extracted_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

In [ ]:
import sqlite3
import pandas as pd

# Find the database file in the extracted directory
db_file = [f for f in os.listdir(extracted_dir) if f.endswith('.sqlite') or f.endswith('.db')][0]
db_file_path = os.path.join(extracted_dir, db_file)

# Connect to the SQLite database
conn = sqlite3.connect(db_file_path)

In [ ]:
# Execute the query and fetch all table names
table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:", table_names['name'].tolist())

# Extract each table into a DataFrame
tables = {}
for table_name in table_names['name']:
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    tables[table_name] = df
    print(f"Loaded table '{table_name}' with {df.shape[0]} rows and {df.shape[1]} columns.")

# Close the connection
#conn.close()

In [ ]:
table_names['name'].tolist()

In [ ]:
aircraft_data = tables['aircrafts_data']

In [ ]:
aircraft_data['model'] = aircraft_data['model'].apply(lambda x: json.loads(x)['en'])

In [ ]:
aircraft_data

In [ ]:
airports_data  = tables['airports_data']

In [ ]:
airports_data['airport_name'] = airports_data['airport_name'].apply(lambda x: json.loads(x)['en'])
airports_data['city'] = airports_data['city'].apply(lambda x: json.loads(x)['en'])

In [ ]:
airports_data

In [ ]:
boarding_passes = tables['boarding_passes']
boarding_passes

In [ ]:
bookings = tables['bookings']
bookings

In [ ]:
flights = tables['flights']
flights

In [ ]:
seats = tables['seats']
seats

In [ ]:
ticket_flights = tables['ticket_flights']
ticket_flights

In [ ]:
tickets = tables['tickets']
tickets

* Checking for missing value

In [ ]:
for table in table_names['name'].tolist():
    print('\ntable:', table)
    df_table = pd.read_sql_query(f"select * from {table}", conn)
    print(df_table.isnull().sum())

## Basic Analysis

#### How many planes have more than 100 seats?

In [ ]:
seat_count = pd.read_sql_query("""select aircraft_code, count(*) as Number_of_seats from seats
                        group by aircraft_code Having Number_of_seats >100;""",conn)
seat_count

In [ ]:
ax = sns.barplot(x = 'aircraft_code', y ='Number_of_seats',
            order = seat_count.sort_values('Number_of_seats', ascending =False)['aircraft_code'], 
            palette ='coolwarm',data  = seat_count)
for container in ax.containers:
    ax.bar_label(container)
plt.xlabel('Aircraft', fontsize =16)
plt.ylabel('Number_of_seats', fontsize =16)
plt.grid('b')
plt.show()

#### Represent variation in number of seats booked and total amount earned with time( per day)

In [ ]:
booked_tickets = pd.read_sql_query("""select * from tickets as t
                     inner join bookings as b on t.book_ref = b.book_ref""",conn)
booked_tickets.dtypes

In [ ]:
import datetime
booked_tickets['book_date'] = pd.to_datetime(booked_tickets['book_date'])
booked_tickets.dtypes

In [ ]:
booked_tickets

In [ ]:
booked_tickets['date']= booked_tickets['book_date'].dt.date
booked_tickets

In [ ]:
booking_count = booked_tickets.groupby(['date'])[['ticket_no']].count()

In [ ]:
plt.figure(figsize = (18,6))
plt.plot(booking_count.index, booking_count['ticket_no'], marker ="*")
plt.xlabel('Date', fontsize = 16)
plt.ylabel('Number of Tickets Booked', fontsize =16)
plt.grid('b')
plt.show()

In [ ]:
amount_per_day = booked_tickets.groupby(['date'])[['total_amount']].sum()

In [ ]:
plt.figure(figsize = (18,6))
plt.plot(amount_per_day.index, amount_per_day['total_amount'], marker ="*")
plt.xlabel('Date', fontsize = 16)
plt.ylabel('Total Amount', fontsize =16)
plt.grid('b')
plt.show()

#### Calculate the avearge charges  for each aircraft with different conditions

In [ ]:
x = pd.read_sql_query(""" Select aircraft_code,fare_conditions, avg(amount) as avg_amount from ticket_flights as tf
                    inner join flights as f on f.flight_id = tf.flight_id
                    group by aircraft_code,fare_conditions""",conn)
x

In [ ]:
plt.figure(figsize =(12,6))
ax =sns.barplot(x ='aircraft_code',y = 'avg_amount', hue = 'fare_conditions', data = x, palette = 'plasma')
plt.xlabel('Aircraft', fontsize = 16)
plt.ylabel('Average price per seat', fontsize =16)
for container in ax.containers:
    ax.bar_label(container)
sns.set_style('darkgrid')
plt.show()

# Analyzing occupancy rate

#### For each aircraft, calculate the total revenue per year and average revenue per ticket.

In [ ]:
dfa = pd.read_sql_query(""" select aircraft_code, ticket_count,total_revenue, total_revenue/ticket_count as avg_revenue_per_ticket from
                    (select aircraft_code, count(ticket_no) as ticket_count, sum(amount) as total_revenue from ticket_flights as tf
                        join flights as f on tf.flight_id = f.flight_id
                        group by aircraft_code)""",conn)
dfa

In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
ax = sns.barplot(x ='aircraft_code',y ='total_revenue', palette ='plasma', data= dfa)
ax.set_ylabel('Total reveune per year', fontsize =16)
ax.tick_params(axis ='y',width = 2,labelsize = 'large')
ax.set_xlabel('Aircarft', fontsize =16)
ax.grid('b')
for container in ax.containers:
    ax.bar_label(container)
ax1 = ax.twinx()
ax1.plot(dfa['aircraft_code'],dfa['ticket_count'],color = 'r',marker = 'D')
ax1.set_ylabel('Ticket Sold per year', fontsize = 16)
ax1.tick_params(axis = 'y',color ='red',width = 2,labelsize = 'large')


In [ ]:
ax = sns.barplot(x = 'aircraft_code', y ='avg_revenue_per_ticket',
            order = dfa.sort_values('avg_revenue_per_ticket', ascending =False)['aircraft_code'], 
            palette ='coolwarm',data  = dfa)
for container in ax.containers:
    ax.bar_label(container)
plt.xlabel('Aircraft', fontsize =16)
plt.ylabel('Revenue per ticket sold', fontsize =16)
plt.grid('b')
plt.show()

#### Average occupancy per aircaft

In [ ]:
df_or =pd.read_sql_query(""" Select a.aircraft_code, avg(a.seats_count) as booked_seats, (avg(a.seats_count)/b.num_seats)*100 as occupancy_rate
                    from 
                    (Select aircraft_code, f.flight_id, count(*) as seats_count from boarding_passes as bp
                    inner join flights as f on f.flight_id =bp.flight_id
                    group by aircraft_code, f.flight_id) as a
                    inner join
                    (select aircraft_code, count(*) as num_seats from seats
                    group by aircraft_code) as b
                    on a.aircraft_code =b.aircraft_code
                    group by a.aircraft_code""",conn)
df_or

In [ ]:
plt.figure(figsize = (10,6))
ax = sns.barplot(x = 'aircraft_code', y ='occupancy_rate',
            order = df_or.sort_values('occupancy_rate', ascending =False)['aircraft_code'], 
            palette ='coolwarm',data  = df_or)
for container in ax.containers:
    ax.bar_label(container)
plt.xlabel('Aircraft', fontsize =16)
plt.ylabel('Occupancy Rate', fontsize =16)
sns.set_style('darkgrid')
plt.show()

#### What would be the increase in total revenue given that all aircraft registerd 10% higher occupancy rate

In [ ]:
df_or['10%_high_occupany_rate'] = df_or['occupancy_rate']*(1.1)
df_or

In [ ]:
df_rev = pd.read_sql_query("""select aircraft_code, sum(amount) as total_revenue from ticket_flights as tf
                        join flights as f on tf.flight_id = f.flight_id
                        group by aircraft_code""",conn)
df_rev

In [ ]:
df_or['Inc Total Annual Reveune'] = (df_rev['total_revenue']/df_or['occupancy_rate'])*df_or['10%_high_occupany_rate']
df_or['Total Annual Revenue'] = df_rev['total_revenue']
df_or['Increase'] = (df_or['Inc Total Annual Reveune'] - df_or['Total Annual Revenue'])

In [ ]:
df_or

In [ ]:
plt.figure(figsize = (10,6))
ax = sns.barplot(x = 'aircraft_code', y ='Increase',
            order = df_or.sort_values('Increase', ascending =False)['aircraft_code'], 
            palette = 'flare', data  = df_or)
for container in ax.containers:
    ax.bar_label(container)
plt.xlabel('Aircraft', fontsize =16)
plt.ylabel('Revenue Incerased by', fontsize =16)
plt.grid('b')
plt.show()